In [11]:
from Bio.Seq import Seq
import gget

import locusPocus
import allel
import numpy as np 
import pandas as pd

Predict protein structure with alphafold

In [3]:
import malariagen_data
ag3 = malariagen_data.Ag3()
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release/
Data releases available : 3.0
Results cache           : None
Cohorts analysis        : 20220608
Species analysis        : aim_20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 6.1.1
Client location         : England, GB
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [4]:
transcript = 'AGAP006228-RA'

In [5]:
gff = ag3.geneset()

transcript_exons = gff.query("type == 'exon' & Parent == @transcript")
transcript_exons

,contig,source,type,start,end,score,strand,phase,ID,Parent,Name,description
24853,2L,VectorBase,exon,28548433,28548763,NaN,+,NaN,NaN,AGAP006228-RA,AGAP006228-RA-E1,NaN
24855,2L,VectorBase,exon,28549062,28549199,NaN,+,NaN,NaN,AGAP006228-RA,AGAP006228-RA-E2,NaN
24857,2L,VectorBase,exon,28549276,28549818,NaN,+,NaN,NaN,AGAP006228-RA,AGAP006228-RA-E3,NaN
24860,2L,VectorBase,exon,28549888,28550748,NaN,+,NaN,NaN,AGAP006228-RA,AGAP006228-RA-E4,NaN


In [6]:
transcript_seq = np.array([])
for start, end in zip(transcript_exons['start'], transcript_exons['end']):
    
    dna = ag3.genome_sequence(region=f'2L:{start}-{end}').compute().astype(str)
    transcript_seq = np.append(transcript_seq, dna)

transcript_seq = ''.join(transcript_seq)
len(transcript_seq)

1873

In [7]:
start_codon_idx = transcript_seq.find("ATG")
coding_seq = transcript_seq[start_codon_idx:]

In [8]:
len(coding_seq)

1683

In [9]:
protein_seq = str(Seq(coding_seq).translate(to_stop=True))
print(len(protein_seq))
protein_seq

540


'MANSELIVSTGYGPVQGTARTSLYGTGYVSFQGIPYAKPPVGELRFKDPTPPENWTQVLDCTEQCDPCFHFDRRVNKIVGSEDSLRLNIFSKTIKPTKPLPVMVYIYGGGFVEGTSGTELYGPDYLIEKDIVLVTLNYRVGALGFLCCQSPTAGVPGNAGLKDQRLALRWVRDNIASFGGDPSAITLFGHSAGGASVQYHTIADASKNLFQRAIIMSGSTMCSWALTPQRNWPEKLAKAIGWQGEGDEEAALQYLRQASPESIVDHQEKLFGPQEIQEGLLSPFAPTIEPYESEVCFIPRSPFEMSRTAWGNSIDIMIGGTSEEGLILLPKVKPQLPSMLQDPRLFVGNVPFHLKLSLEQRMAFGEQLKQLYYPDSNPSIDNLDGFVNMASDRIFWHDLHRTILARANYACTAKTFVYRFCVDSPFFNHYRIHMVDPNARGTSHADEISYLFSNIFAKPLDKSTLEYRAIQHLVDIFTSFATNSDPNCDSTASLSWTAVPKTAPPYNCLNISNDGVEVVELPESRRLQLWDSFYVNDALF'

In [12]:
clusters_df, cluster_idxs, palette = locusPocus.load_cluster_data()

In [18]:
metahaps = pd.read_csv("../../results/haplotype_metahaps.tsv", sep="\t")

In [38]:
for k,v in cluster_idxs.items():
    if k == 'WT': continue
    
    meta_1 = metahaps.query("hap_cluster == @k")
    meta_1_hom = meta_1[meta_1.duplicated()]
    sample_id = meta_1_hom.iloc[0,0]
    print(k, sample_id)
    
    hap = ag3.haplotypes(
        region="2L:28548433-28550748", 
        analysis='gamb_colu', 
        sample_query=f"partner_sample_id == {sample_id}"
    )

C1 BL0062-C


UndefinedVariableError: name 'BL0062' is not defined

In [33]:
meta_1_hom.iloc[0,0]

'AA0052-C'

In [ ]:
hap